In [1]:
!git clone https://github.com/AminMohamed-3/Emotion-Classification.git
!pip install transformers dataset accelerate -q
import sys
sys.path.append("/kaggle/working/Emotion-Classification")

fatal: destination path 'Emotion-Classification' already exists and is not an empty directory.


In [2]:
import os
import numpy as np
import torch
import wandb
from datasets import load_dataset
# from dotenv import load_dotenv
from transformers import (
    AutoModelForSequenceClassification,
    AutoTokenizer,
    DataCollatorForTokenClassification,
    Trainer,
    TrainingArguments,
)
import yaml

with open("config.yaml", "r") as f:
    config = yaml.safe_load(f)
    NUM_LABELS = config["NUM_LABELS"]
from Training.utils import compute_metrics
from Training.dataset import prepare_simplified_dataset, prepare_local_dataset

# load_dotenv()

# Load & Tokenize dataset

In [3]:
model_checkpoint = "FacebookAI/roberta-base"  # Using a larger model
tokenizer = AutoTokenizer.from_pretrained(model_checkpoint)
LEGO_dataset, id2label, label2id = prepare_local_dataset(tokenizer, train_path = "cleaning/run1/train_filtered.csv", val_path = "cleaning/run1/validation_filtered.csv")

Map:   0%|          | 0/43410 [00:00<?, ? examples/s]

Map:   0%|          | 0/5426 [00:00<?, ? examples/s]

Map:   0%|          | 0/43410 [00:00<?, ? examples/s]

Map:   0%|          | 0/5426 [00:00<?, ? examples/s]

In [4]:
GO_dataset, _, _ = prepare_simplified_dataset(tokenizer)

# Define Model

In [5]:
model = AutoModelForSequenceClassification.from_pretrained(
    model_checkpoint,
    num_labels=NUM_LABELS,
    id2label=id2label,
    label2id=label2id,
    problem_type="multi_label_classification",
)
device = "cuda" if torch.cuda.is_available() else "cpu"
model = model.to(device)

Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at FacebookAI/roberta-base and are newly initialized: ['classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


# Trainer

In [6]:
# Define the training arguments
training_args = TrainingArguments(
    output_dir="./results",
    num_train_epochs=10,  # Training for longer
    per_device_train_batch_size=32,  # smaller batch size
    per_device_eval_batch_size=48,
    evaluation_strategy="epoch",
    save_strategy="epoch",
    learning_rate=2e-5,  # Using a smaller LR
    save_total_limit=10,
    load_best_model_at_end=True,
    metric_for_best_model="f1",
    greater_is_better=True,
    logging_steps=20,
    weight_decay=0.01,
    warmup_ratio=0.1,
    lr_scheduler_type="cosine",
)

In [7]:
import torch

class CustomTrainer(Trainer):
    def compute_loss(self, model, inputs, return_outputs=False):
        # Move the labels to the same device as the model's parameters
        labels = inputs.pop("labels").to(torch.float32)
        # Forward pass
        outputs = model(**inputs)
        # Compute the loss
        loss = torch.nn.functional.binary_cross_entropy_with_logits(
            outputs.logits, labels
        )
        return (loss, outputs) if return_outputs else loss

In [9]:
trainer = CustomTrainer(
    model=model,
    args=training_args,
    train_dataset=LEGO_dataset["train"],
    eval_dataset=LEGO_dataset["validation"],
    compute_metrics=compute_metrics,
)

Detected kernel version 5.4.0, which is below the recommended minimum of 5.5.0; this can cause the process to hang. It is recommended to upgrade the kernel to the minimum version or higher.


In [10]:
wandb.login(key="a4248310ebed11864fc97c90d419f4d4a6331a9a")
trainer.train()

Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: Currently logged in as: aminmoe707. Use `wandb login --relogin` to force relogin
wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: Appending key for api.wandb.ai to your netrc file: /home/user/.netrc
wandb: WARNING The `run_name` is currently set to the same value as `TrainingArguments.output_dir`. If this was not intended, please specify a different run name by setting the `TrainingArguments.run_name` parameter.


Epoch,Training Loss,Validation Loss


KeyboardInterrupt: 